In [1]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *
from netgen.geom2d import SplineGeometry
import sys
sys.path.insert(0,"../")
import problems
import numpy as np
import params
import pickle
from time import time
from scipy.optimize import minimize_scalar
import scipy as sp



In [2]:
## Check positivity
def check_pos_gfu(gfu,mesh):
    vertices =( (0,0), (90,0))
    m1 = mesh(vertices[0])
    m2 = mesh(vertices[1])
    print(gfu(m1),gfu(m2))



In [2]:
w,l= 3,15
n = 5
rect = SplineGeometry()
pnts = [(0,0), (l,0), (l,w), (0,w)]
p1,p2,p3,p4 = [rect.AppendPoint(*pnt) for pnt in pnts]
curves = [[["line",p1,p2],"bottom"],
        [["line",p2,p3],"right"],
        [["line",p3,p4],"top"],
        [["line",p4,p1],"left"]]
[rect.Append(c,bc=bc, leftdomain=1, rightdomain=0,maxh=0.05) for c,bc in curves]


mesh = Mesh(rect.GenerateMesh(maxh=0.5))
print("meshed")
Draw(mesh)

meshed


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [3]:
## get problem parameters and geometry
from ngsolve.meshes import MakeStructured2DMesh
problem = problems.problem2

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = "../"+problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 0.7
ord = 3
ord_phi = ord - 3
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional

rho = 1.23e-6


phi = lambda J: phi0/J

G = Parameter(G_target)

# Generate mesh and geometry ### add parallel stuff
# def mesher(geom, h):
#     if ".stp" in geom:
#         geo = OCCGeometry(geom)
#     else:
#         geo = pickle.load(open(geom, "rb"))

#     mesh = Mesh(geo.GenerateMesh(maxh=h))
#     return mesh
# mesh = mesher(geom, h)

# l,w = 15,3
# mapper = lambda x,y: (x*l,y*w)
# mesh = MakeStructured2DMesh( mapping=mapper, nx=25, ny=5)





def Gel_energy_functional(F,u):
    g = 9.81
    J = Det(F)
   
    H = (J - phi0)*log(1-phi0/J)  + phi0 * chi*(1-phi0/J)
    # calculate C
    # min (G/2) * (3 lam**2) + KBTV * H(lam**3):  lam > 1
    Hf = lambda lam: (lam-phi0)*np.log(1-phi0/lam) + phi0*chi*(1-phi0/lam)

    fun = lambda lam: (G.Get()/2)*(3*lam**2) + KBTV*Hf(lam)
    res = minimize_scalar(fun, bounds=(1, 10), method='bounded')
    C = res.fun
    return 0.5*(G)* Trace(F.trans*F ) + H*KBTV + rho*g*u[1] - C

## Generate spaces and forms
"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
if BC["dir_cond"] == "faces":
    fesu = VectorH1(mesh, order=ord, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fesu = VectorH1(mesh, order=ord, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])

fesphi = L2(mesh, order = ord_phi)

u, v = fesu.TnT()
delta, vphi = fesphi.TnT()
# Assemble all bilinear forms

## Assemble forms
alpha = Parameter(1)
psih = GridFunction(fesphi)
psih.Set(1)

eps_i = 1e-3
uk = GridFunction(fesu)

psik = GridFunction(fesphi)
psik.Set(1e-2)
eps = 1e-6


obs = CF(-y)


def Assembler_A(fesu):
    BF = BilinearForm(fesu)
    F = Id(2)+ grad(u)
    BF += Variation(alpha * Gel_energy_functional(F,u).Compile()*dx)
    return BF


def Assembler_B(trialspace= fesphi, testspace=fesu):
    BF = BilinearForm(trialspace, testspace)
    BF += delta * v[1]*dx
    # BF += u[1]*vphi*dx??
    return BF

def Assembler_C(fesphi,psih):
    BF = BilinearForm(fesphi)
    if ord_phi == 0:
        BF+= -delta * exp(psih) * vphi * dx - eps * (delta * vphi * dx)
    else:
        BF+= -delta * exp(psih) * vphi * dx - eps * (grad(delta) * grad(vphi) * dx)
    return BF

def Assembler_L1(psih, psik, fes):
    LF = LinearForm(fes)
    LF += (psik - psih)*v[1]*dx
    return LF
    
def Assembler_L2(fesphi,psih):
    LF = LinearForm(fesphi)
    LF += (obs + exp(psih))*vphi*dx
    return LF
    

## A###
A = Assembler_A(fesu)
fd = np.nonzero(A.space.FreeDofs())[0]
### B ####
B = Assembler_B(trialspace=fesphi, testspace=fesu)
B.Assemble()
B_rows,B_cols,B_vals = B.mat.COO()
B_mat_full = sp.sparse.csr_matrix((B_vals, (B_rows, B_cols)), shape=(fesu.ndof, fesphi.ndof))
B_mat = B_mat_full[fd,:]
## C ###
C = Assembler_C(fesphi,psih)

## L1 ###
L1 = Assembler_L1(psih,psik,fesu)

## L2 ###
L2 = Assembler_L2(fesphi,psih)


# define phi as the 0 function 

gfu = GridFunction(fesu)
scenes = GridFunction(fesu, multidim = 0)

In [4]:

def Solve_schur(A_inv,A_mat,B_mat,F2,C,L1,L2,res = None,fd=None):
        """
        All in ngsolve format and assembled
        """
        C_rows,C_cols,C_vals = C.mat.COO()
        C_mat = sp.sparse.csr_matrix((C_vals, (C_rows, C_cols)), shape=C.mat.shape)
        #C_inv = sp.sparse.linalg.inv(C_mat)
    
        # get vectors in numpy form
        L1_vec_full = L1.vec.FV().NumPy()
        L1_vec = L1_vec_full[fd]
        
        if res:
            res_n_vec = res.FV().NumPy()[fd]

        L2_vec = L2.vec.FV().NumPy()

        # schur complement for u_it
        # F1 = B_mat @ C_inv @ B_mat.T
        # get schur complement inverse
        # schur_u = A_inv - F

        

        schur_delta_inv = sp.sparse.linalg.inv(C_mat - F2) #calc stuff out of iteration and replace delta after calculations for u
        delta_vec = schur_delta_inv @ (L2_vec - B_mat.T @ A_inv @ (L1_vec + res_n_vec)) # solve for delta
        u_vec = np.zeros(len(L1_vec_full))
        u_vec[fd] = A_inv @ (L1_vec + res_n_vec - B_mat @ delta_vec) # solve for u_it
        return u_vec, delta_vec

def get_alpha(alpha, k,type="geom"):
    if type == "exp":
        return 2**k
    elif type == "geom":
        return min([max(1,1.5**(1.5**(k))-alpha.Get()), 1e10])
    elif type == None:
        return 1

      

In [5]:
import scipy as sp
max_PG_it = 100
max_iter_newton = 100
max_iter_qnewton = 100
newton_damp = 0.3
softening_n = 40

tol_newton = 1e-7
tol_QN = 1e-7
tol_PG = 1e-7
tol_delta = 1e-7
gammas = np.flip(np.linspace(G_target, G_target*15 ,softening_n))
first_bit = gammas[:-1]
second_bit = np.flip(np.linspace(G_target,gammas[-2], 5))
gammas = np.concatenate((first_bit,second_bit))

u_n = GridFunction(fesu)

for num, load in enumerate(gammas): # softening
    G.Set(load) #incremental softening
    print("Softening step:",num, "Load:",load)

    n = 0
    while n < max_iter_newton: # Proximal Galerkin
        print("Newton it:",n)
        
        u_n.vec.data = gfu.vec
        res_n = gfu.vec.CreateVector()
        A.Apply(u_n.vec,res_n)
        A.AssembleLinearization(u_n.vec)
        
        A_rows,A_cols,A_vals = A.mat.COO()
        A_mat_full = sp.sparse.csr_matrix((A_vals, (A_rows, A_cols)), shape=A.mat.shape)
        A_mat = A_mat_full[fd,:][:,fd]
        A_inv = sp.sparse.linalg.inv(A_mat)
        F2 = B_mat.T @ A_inv @ B_mat
        #A_inv = A.mat.Inverse(freedofs= fesu.FreeDofs())
        
        k = 0
        while k < max_PG_it: #PG
            print("PG it:",k)
            alpha_num = get_alpha(alpha, k,type="exp")
            alpha.Set(alpha_num)


            psik.vec.data = psih.vec
            if k != 0:
                uk.vec.data = u_it.vec # update u_k to compare later
            u_it = GridFunction(fesu)
            d_it = GridFunction(fesphi)
            for it in range(max_iter_qnewton): #QN
                with TaskManager():
                    L1 = Assembler_L1(psih,psik,fesu)
                    L1.Assemble()
                    L2 = Assembler_L2(fesphi,psih)
                    L2.Assemble()
                    C = Assembler_C(fesphi,psih)
                    C.Assemble()
                # get free dofs

                # get scipy sparse matrix

                u_vec , delta_vec = Solve_schur(A_inv,A_mat,B_mat,F2,C,L1,L2,res_n,fd)

                # update grid function
                u_it.vec.FV().NumPy()[:] = u_vec
                d_it.vec.FV().NumPy()[:] = delta_vec
                qn_val = Integrate(d_it**2, mesh)
                
                psih.vec.data = psih.vec + d_it.vec
                if abs(qn_val) < tol_QN:
                    print("QN converged",qn_val)
                    break
               
            
                
            pg_val = Integrate((u_it-uk)**2, mesh)
            if pg_val < tol_PG:
                print("PG converged",pg_val)
                break
            k += 1
        # newton update
        
        gfu.vec.data = gfu.vec - newton_damp*u_it.vec
        newton_val = Integrate((gfu-u_n)**2, mesh)
        if newton_val < tol_newton:
            print("Newton converged",   newton_val)
            break
        n += 1
Draw(gfu, mesh, "gfu", redraw=True)

Softening step: 0 Load: 1.9500000000000002
Newton it: 0


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:597: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)


: 

In [ ]:
Draw(gfu, mesh, "gfu", redraw=True)

In [ ]:
Draw(obs+exp(psih), mesh, "obs", redraw=True)

In [ ]:
gfu_tilde = GridFunction(fesu)
gfu_tilde.vec.data = gfu.vec
gfu_tilde.components[1].Set( exp(psih)+obs)
Draw(gfu_tilde, mesh, "gfu_tilde", redraw=True)

Intro: noción física del problema,Flory-Huggins and non linear,  por que nos interesa el problema de contacto, deducción de la energía
Hacer dibujo para gel bonded flat
Hacer dibujo para gel bonded con geometrias curvas o generales
Formulación matemátcia del problema ( con elementos finitos), escribir formulación variacional, problema de contacto, el metodo no funciona bien, comparar sin penalty,con penalty. Mostrar incremental softening.

Proximal Galerkin Referencia brendan keith, no se ha aplicado a elasticidades lineales. Mostrar resultados para elasticidad lineal.
Dificultades de PG (2 problemas no lineales) comparar como se implementa antes y ahora.

BIBLIOGRAFÍA
agregar acknowledgement a FONDECYT.


In [ ]:
mip = mesh(0,0.2)
gfu(mip)

probar penalty, refinamiento de malla, proyectar u a al espacio de varphi, probar cuadrilateros.

In [ ]:
prepara presentación talca 20 min beamer